# Introduction to MPI on Amazon SageMaker

Message Passing Interface (MPI) is the fundamental communication protocol for programming parallel computer programs. See its [wiki page](https://en.wikipedia.org/wiki/Message_Passing_Interface). [Open MPI](https://www.open-mpi.org/projects/user-docs/) is the implementation that's used as a basic building block for distributed training systems. 

In Python programs, you can interact with Open MPI APIs via [mpi4py](https://mpi4py.readthedocs.io/en/stable/overview.html) and easily convert your single-process python program into a parallel python program. 

Parallel processes can exist on one host (e.g. one EC2 instance) or multiple hosts (e.g. many EC2 instances). It's trivial to set up a parallel cluster (comm world, in MPI parlance) on one host via Open MPI, but it is less straight-forward to set up an MPI comm world across multiple instances. 

SageMaker does it for you. In this tutorial, you will go through a few basic (but exceeding important) [MPI communications](https://mpi4py.readthedocs.io/en/stable/tutorial.html) on SageMaker with **multiple instances** and you will verify that parallel processes across instances are indeed talking to each other. Those basic communications are the fundamental building blocks for distributed training.

## Environment 
We assume Open MPI and mpi4py have been installed in your environment. This is the case for SageMaker Notebook Instance or Studio. 

## Inspect the Python Program

In [1]:
!pygmentize mpi_demo.py

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    print("Number of MPI processes that will talk to each other:", size)


def point_to_point():
    """Point to point communication
    Send a numpy array (buffer like object) from rank 0 to rank 1
    """
    if rank == 0:
        print("point to point")
        data = np.array([0, 1, 2], dtype=np.intc)  # int in C

        # remember the difference between
        # Upper case API and lower case API
        # Basically uppper case API directly calls C API
        # so it is fast
        # checkout https://mpi4py.readthedocs.io/en/stable/

        comm.Send([data, MPI.INT], dest=1)
    elif rank == 1:
        print(f"Hello I am rank {rank}")
        data = np.empty(3, dtype=np.intc)
        comm.Recv([data, MPI.INT], source=0)
        print("I received some data:", data)

    if rank == 0:
        time.sleep(1)  # give some buffer time for executio

See the program in action with 2 parallel processes on your current environment. Make sure you have at least 2 cores.

In [2]:
!mpirun -np 2 python mpi_demo.py

Hello I am rank 1
Number of MPI processes that will talk to each other: 2
point to point
I received some data: [0 1 2]
Broadcasting from rank 0
Data at rank 0 [0 1 2 3 4 5 6 7 8 9]
Data at rank 1 [0 1 2 3 4 5 6 7 8 9]
Gather and reduce
I am rank 0, data I gathered is: [[0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]]
I am rank 0, my avg is: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
I am rank 1, my avg is: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
mpi_demo.py:83: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  avg = np.mean(recvbuf, axis=0, dtype=np.float)
mpi_demo.py:89: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this war

## Scale it on SageMaker
You can run the above program with $n$ processes per host across $N$ hosts on SageMaker (and get a comm world of size $n\times N$). In the remaining of this notebook, you will use SageMaker TensorFlow deep learning container to run the above program. There is no particular reason for the choice, all SageMaker deep learning containers have Open MPI installed. So feel free to replace it with your favorite DLC. 

Check out the [SageMaker Python SDK Docs](https://sagemaker.readthedocs.io/en/stable/api/training/smd_model_parallel_general.html?highlight=mpi%20paramters#mpi-parameters) for the parameters needed to set up a distributed training job with MPI. 

In [7]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.pytorch import PyTorch

#role = get_execution_role()
role = 'arn:aws:iam::976939723775:role/service-role/AmazonSageMaker-ExecutionRole-20210317T133000'
# Running 2 processes per host
# if we use 3 instances,
# then we should see 6 MPI processes

distribution = {"mpi": {"enabled": True, "processes_per_host": 2}}

tfest = TensorFlow(
    entry_point="mpi_demo.py",
    role=role,
    framework_version="2.3.0",
    distribution=distribution,
    py_version="py37",
    instance_count=3,
    instance_type="ml.c5.2xlarge",  # 8 cores
    output_path="s3://" + sagemaker.Session().default_bucket() + "/" + "tf_mpi",
)

torchfest = PyTorch(
    entry_point="mpi_demo.py",
    role=role,
    framework_version="1.10.0",
    distribution=distribution,
    py_version="py38",
    instance_count=3,
    instance_type="ml.c5.2xlarge",  # 8 cores
    output_path="s3://" + sagemaker.Session().default_bucket() + "/" + "torch_mpi",
)

In [5]:
tfest.fit(wait=False)

2022-07-06 17:54:31 Starting - Starting the training job...
2022-07-06 17:54:56 Starting - Preparing the instances for trainingProfilerReport-1657130070: InProgress
.........
2022-07-06 17:56:26 Downloading - Downloading input data
2022-07-06 17:56:26 Training - Downloading the training image...
2022-07-06 17:56:56 Training - Training image download completed. Training in progress..2022-07-06 17:57:00,835 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-07-06 17:57:00,842 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-06 17:57:01,206 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-06 17:57:01,221 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-06 17:57:01,229 sagemaker-training-toolkit INFO     Starting MPI run as worker node.
2022-07-06 17:57:01,229 sagemaker-training-toolkit INFO     Waiting for MPI Master

In [9]:
torchfest.fit(wait=True)

2022-07-06 18:50:57 Starting - Starting the training job...
2022-07-06 18:51:22 Starting - Preparing the instances for trainingProfilerReport-1657133457: InProgress
.........
2022-07-06 18:52:42 Downloading - Downloading input data...
2022-07-06 18:53:22 Training - Downloading the training image...
2022-07-06 18:53:55 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-06 18:53:57,030 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-06 18:53:57,031 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-06 18:53:57,038 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-06 18:53:57,044 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-06 18:53:57,471 sagemaker-training-toolkit INFO     No G

UnexpectedStatusException: Error for Training job pytorch-training-2022-07-06-18-50-57-442: Failed. Reason: AlgorithmError: Framework Error: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_training/trainer.py", line 87, in train
    entrypoint()
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_pytorch_container/training.py", line 121, in main
    train(environment.Environment())
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_pytorch_container/training.py", line 68, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_training/entry_point.py", line 99, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_training/process.py", line 279, in run
    self._setup()
  File "/opt/conda/lib/python3.8/site-packages/sagemaker_training/mpi.py", line 160, in _setup
    _st

The stdout "Number of MPI processes that will talk to each other:  6" indicates that the processes on all hosts are included in the comm world. 

## Conclusion
In this notebook, you went through some fundamental MPI operations, which are the bare bones of inner workings of many distributed training frameworks. You did that on SageMaker with multiple instances. You can scale up this set up to include more instances in a real ML project.